In [1]:
import matplotlib.pyplot as plt
import random
import torch
from attack import (
    reconstruct_interactions,
    reconstruct_interactions_with_private_params,
)
from ranker import (
    LinearPDGDRanker,
    Neural1LayerPDGDRanker,
    Neural2LayerPDGDRanker,
    CollaborativeFilteringRecommender,
)
from tqdm import tqdm
from utils import Metrics

In [33]:
# Simulation for LTR

torch.manual_seed(2023)
random.seed(2023)

num_sim_round = 1
num_features = 100
num_data = 1000
lr = 1e-01
max_iters = 1000
num_atk = 1

metrics = Metrics()

models = {
    "linear_pdgd": LinearPDGDRanker(num_features),
    # "neural_1_pdgd": Neural1LayerPDGDRanker(num_features, hidden_size=5),
    # "neural_2_pdgd": Neural2LayerPDGDRanker(
    #     num_features, hidden_size=4, hidden_size2=2
    # ),
}

for _ in tqdm(range(num_sim_round)):
    features = torch.rand(num_data, num_features) * 2 - 1
    interactions = torch.randint(0, 2, (num_data,))
    while interactions.sum() == 0:
        interactions = torch.randint(0, 2, (num_data,))
    ranking = list(range(num_data))
    random.shuffle(ranking)
    ranking = torch.LongTensor(ranking)

    for model_name, model in models.items():
        params = model.gen_params()
        log_pos_bias_weight = model.calc_log_pos_bias_weight(
            ranking, model.forward_multiple(params, features), num_data
        )
        
        target = model.grad(
            params,
            features,
            ranking,
            interactions,
            log_pos_bias_weight=log_pos_bias_weight,
        )

        preds_raw = reconstruct_interactions(
            lambda I: model.grad(
                params, features, ranking, I, log_pos_bias_weight=log_pos_bias_weight
            ),
            target,
            num_data,
            lr=lr,
            max_iters=max_iters,
            num_rounds=num_atk,
            return_raw=True,
        )
        preds = preds_raw.sigmoid().round().long()

        metrics.update(model_name, interactions, preds, preds_raw=preds_raw)

100%|██████████| 1/1 [00:49<00:00, 49.41s/it]


In [34]:
print(metrics.get_dataframe().to_string())

          name  accuracy        f1  precision    recall       auc   auc-pr  extra_data
0  linear_pdgd     0.591  0.590591       0.59  0.591182  0.651251  0.65025         NaN


In [27]:
# Simulation for collaborative filtering

torch.manual_seed(2023)
random.seed(2023)

num_sim_round = 10
num_features = 64
num_data = 1000
lr = 1e-01
max_iters = 100000
num_atk = 10

metrics = Metrics()

for _ in tqdm(range(num_sim_round)):
    features = torch.rand(num_data, num_features) * 2 - 1
    user_embedding = torch.rand(num_features) * 2 - 1

    interactions = torch.randint(0, 2, (num_data,))
    while interactions.sum() == 0:
        interactions = torch.randint(0, 2, (num_data,))

    cf_rec = CollaborativeFilteringRecommender()
    target = cf_rec.federated_item_grad(user_embedding, features, interactions)

    user_embedding2 = torch.rand(num_features) * 2 - 1
    preds_raw = reconstruct_interactions(
        lambda I: cf_rec.federated_item_grad(user_embedding2, features, I),
        target,
        num_data,
        lr=lr,
        max_iters=max_iters,
        num_rounds=num_atk,
        return_raw=True,
    )
    preds = preds_raw.sigmoid().round().long()

    metrics.update("FCF_simple", interactions, preds, preds_raw=preds_raw)

    preds_raw, user_embedding_est = reconstruct_interactions_with_private_params(
        lambda I, U: cf_rec.federated_item_grad(U, features, I),
        target,
        num_data,
        num_features,
        lr=lr,
        max_iters=max_iters,
        num_rounds=num_atk,
        return_raw=True,
    )
    preds = preds_raw.sigmoid().round().long()
    
    metrics.update("FCF_private", interactions, preds, preds_raw=preds_raw)

100%|██████████| 10/10 [00:40<00:00,  4.09s/it]


In [28]:
print(metrics.get_dataframe().to_string())

           name  accuracy        f1  precision    recall       auc    auc-pr  extra_data
0    FCF_simple      0.43  0.359551   0.380952  0.340426  0.398234  0.416265         NaN
1   FCF_private      1.00  1.000000   1.000000  1.000000  1.000000  1.000000         NaN
2    FCF_simple      0.52  0.500000   0.545455  0.461538  0.542869  0.555989         NaN
3   FCF_private      1.00  1.000000   1.000000  1.000000  1.000000  1.000000         NaN
4    FCF_simple      0.42  0.355556   0.444444  0.296296  0.439614  0.523032         NaN
5   FCF_private      1.00  1.000000   1.000000  1.000000  1.000000  1.000000         NaN
6    FCF_simple      0.52  0.500000   0.558140  0.452830  0.543557  0.587922         NaN
7   FCF_private      1.00  1.000000   1.000000  1.000000  1.000000  1.000000         NaN
8    FCF_simple      0.45  0.337349   0.400000  0.291667  0.446314  0.462746         NaN
9   FCF_private      1.00  1.000000   1.000000  1.000000  1.000000  1.000000         NaN
10   FCF_simple      